In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from cleaner import profiles, heats, scores
import pandas as pd
import numpy as np

2024-04-02 09:03:21.202016: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 09:03:21.206273: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 09:03:21.268099: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-02 09:03:22.378919: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Index(['Name', 'Nation', 'Followers', 'Age', 'Birthdate', 'Height', 'Weight',
       'Hometown', 'First Season', 'Stance'],
      dtype='object')


In [2]:
# TODO: going to make a regression DL model that predicts 
# a surfers score based on the heat conditions and the surfer?
# Or even a classification model that predicts the probabilitry of a surfer winnign or losing
# based on heat conditions and who the surfer is, or even the surfer details.

In [3]:
profiles["surfer_id"] = profiles["Name"].str.replace(" ", "").str.lower()+profiles["Nation"].str.lower().str.replace(" ", "")

In [4]:
combined_data = heats.merge(scores, on="heat_id").drop_duplicates()

In [5]:
combined_data = combined_data.merge(profiles, on="surfer_id").drop_duplicates()
combined_data["round_name"] = combined_data["round_name"].replace({"Quarterfinals":"Quarterfinal", "Semifinals":"Semifinal"})

In [6]:
def is_max_heat(row):
    max_heat = combined_data[combined_data['heat_id'] == row['heat_id']]['heat_total'].max()
    return 1 if row['heat_total'] == max_heat else 0

combined_data['won'] = combined_data.apply(is_max_heat, axis=1)

In [7]:
target = combined_data["won"].values
target = to_categorical(target)

In [8]:
predictors_pd = combined_data[
       [
       'round_name',
       'event_group_name', 
       'wave_range', 
       'wind',
       'Name', 
       'Nation',
       'Followers', 
       'Age', 
       'Birthdate', 
       'Height', 
       'Weight', 
       'Hometown', 
       'Stance'
       ]
       ]

In [9]:
quant_predictors = predictors_pd[["Height", "Weight", "Age", "Followers"]].values

In [105]:
combined_data["heat_total"] = combined_data["heat_total"].replace('&ndash;&ndash;', np.nan).astype(float)

In [10]:
combined_data["round_name_factor"], _ = pd.factorize(combined_data["round_name"])
combined_data["event_group_name_factor"], _ = pd.factorize(combined_data["event_group_name"])
combined_data["wave_range_factor"], _ = pd.factorize(combined_data["wave_range"])
combined_data["wind_factor"], _ = pd.factorize(combined_data["wind"])
combined_data["Name_factor"], _ = pd.factorize(combined_data["Name"])
combined_data["Nation_factor"], _ = pd.factorize(combined_data["Nation"])
combined_data["Hometown_factor"], _ = pd.factorize(combined_data["Hometown"])
combined_data["Stance_factor"], _ = pd.factorize(combined_data["Stance"])

In [11]:
round_name = to_categorical(combined_data["round_name_factor"])
event_group_name = to_categorical(combined_data["event_group_name_factor"])
wave_range = to_categorical(combined_data["wave_range_factor"])
wind = to_categorical(combined_data["wind_factor"])
name = to_categorical(combined_data["Name_factor"])
nation = to_categorical(combined_data["Nation_factor"])
hometown = to_categorical(combined_data["Hometown_factor"])
stance = to_categorical(combined_data["Stance_factor"])

In [12]:
predictors = np.column_stack(
    (
        # quant_predictors,
        # round_name, 
        event_group_name, 
        wave_range,
        # wind,
        name,
        # nation,
        # hometown,
        # stance
    )
                )

In [13]:
nan_indices = np.isnan(predictors).any(axis=1)
predictors = predictors[~nan_indices]

In [14]:
input_shape = predictors.shape[1]

In [21]:
model = Sequential()

model.add(Dense(250, activation="relu", input_shape=(input_shape,)))
model.add(Dense(250, activation="relu"))
model.add(Dense(250, activation="relu"))
model.add(Dense(250, activation="relu"))
model.add(Dense(250, activation="relu"))

model.add(Dense(2, activation="softmax"))

In [22]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [23]:
early_stop_callback = EarlyStopping(patience=5)

model.fit(predictors, target, epochs=40, callbacks=[early_stop_callback], validation_split=0.3)

Epoch 1/40
113/113 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5458 - loss: 0.6906 - val_accuracy: 0.6156 - val_loss: 0.6636
Epoch 2/40
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6317 - loss: 0.6490 - val_accuracy: 0.5916 - val_loss: 0.6673
Epoch 3/40
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6456 - loss: 0.6309 - val_accuracy: 0.6104 - val_loss: 0.6834
Epoch 4/40
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6725 - loss: 0.5966 - val_accuracy: 0.6195 - val_loss: 0.6727
Epoch 5/40
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6865 - loss: 0.5780 - val_accuracy: 0.5890 - val_loss: 0.7241
Epoch 6/40
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7107 - loss: 0.5430 - val_accuracy: 0.6039 - val_loss: 0.7917


In [90]:
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten
from tensorflow.keras.models import Model
import numpy as np

# Model 1 Surfer Embeddings effect on winning a heat
surfer_input = Input((2,), name="Surfer-Name-Input")
n_surfers = np.unique(combined_data["Name"]).shape[0]
surfer_embed = Embedding(input_dim=n_surfers, output_dim=2, name="Surfer-Strength")

embedded_input = surfer_embed(surfer_input)

surfer_strength = Flatten()(embedded_input)

surfer_strength_hidden = Dense(100, activation="relu")(surfer_strength)

surfer_strength_output = Dense(1, activation="sigmoid")(surfer_strength_hidden)

embed_surfer = Model(surfer_input, surfer_strength_output, name="Surfer-Strength-Model")

embed_surfer.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

combined_data["numeric_name"] = combined_data["Name"].map(dict(zip(np.unique(combined_data["Name"]), range(1, len(np.unique(combined_data["Name"]))+1))))
combined_data["numeric_event"] = combined_data["event_group_name"].map(dict(zip(np.unique(combined_data["event_group_name"]), range(1, len(np.unique(combined_data["event_group_name"]))+1))))

early_stop_callback = EarlyStopping(patience=5)

embed_surfer.fit(combined_data[["numeric_name", "numeric_event"]], combined_data["won"], epochs=100, callbacks=[early_stop_callback], validation_split=0.3)

Epoch 1/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - accuracy: 0.5519 - loss: 0.6882 - val_accuracy: 0.5773 - val_loss: 0.6772
Epoch 2/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5760 - loss: 0.6743 - val_accuracy: 0.6351 - val_loss: 0.6592
Epoch 3/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6318 - loss: 0.6512 - val_accuracy: 0.6292 - val_loss: 0.6551
Epoch 4/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6299 - loss: 0.6384 - val_accuracy: 0.6279 - val_loss: 0.6552
Epoch 5/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6356 - loss: 0.6341 - val_accuracy: 0.6240 - val_loss: 0.6562
Epoch 6/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6459 - loss: 0.6305 - val_accuracy: 0.6143 - val_loss: 0.6569
Epoch 7/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6391 - loss: 0.6288 - val_accuracy: 0.6214 - val_loss: 0.6566
Epoch 8/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6303 - loss: 0.6402 - val_acc

In [92]:
combined_data["strength"] = embed_surfer.predict(combined_data[["numeric_name", "numeric_event"]])

161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [93]:
combined_data[["Name", "event_group_name", "strength"]].sort_values(by="strength", ascending=False).drop_duplicates().head(50)

,Name,event_group_name,strength
3883,Gabriel Medina,Tahiti,0.718031
2144,Gabriel Medina,Peniche Portugal,0.715556
506,Gabriel Medina,Pipe Masters,0.714302
4219,Gabriel Medina,G-Land Pro,0.713606
2486,Gabriel Medina,France,0.710850
1195,Gabriel Medina,Surf Ranch,0.708757
2096,Gabriel Medina,Bali Pro,0.708739
3266,Gabriel Medina,Narrabeen (formerly Sydney Coke Classic),0.708634
650,Gabriel Medina,Gold Coast,0.708416
4859,Gabriel Medina,World Cup/Sunset,0.706135


In [119]:
###### Train regression model

# event input
# surfer input
# score regression output

surfer_input = Input((2,), name="Surfer-Name-Input")
n_surfers = combined_data[["Name", "event_group_name"]].drop_duplicates().values.shape[0]
surfer_embed = Embedding(input_dim=n_surfers, output_dim=1, name="Surfer-Strength")

embedded_input = surfer_embed(surfer_input)

surfer_strength = Flatten()(embedded_input)

surfer_strength_hidden = Dense(100, activation="tanh")(surfer_strength)

surfer_strength_output = Dense(1)(surfer_strength_hidden)

embed_surfer = Model(surfer_input, surfer_strength_output, name="Surfer-Strength-Model")

embed_surfer.compile(optimizer="adam", loss="mean_absolute_error")

combined_data["numeric_name"] = combined_data["Name"].map(dict(zip(np.unique(combined_data["Name"]), range(1, len(np.unique(combined_data["Name"]))+1))))
combined_data["numeric_event"] = combined_data["event_group_name"].map(dict(zip(np.unique(combined_data["event_group_name"]), range(1, len(np.unique(combined_data["event_group_name"]))+1))))

early_stop_callback = EarlyStopping(patience=5)

embed_surfer.fit(combined_data.dropna()[["numeric_name", "numeric_event"]], combined_data.dropna()["heat_total"], epochs=100, callbacks=[early_stop_callback], validation_split=0.3)

Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 10.7662 - val_loss: 6.9633
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.8396 - val_loss: 2.5942
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.4773 - val_loss: 2.5357
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.4261 - val_loss: 2.4858
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.4423 - val_loss: 2.4667
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.3293 - val_loss: 2.4534
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2942 - val_loss: 2.4495
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.3227 - val_loss: 2.4477
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.3656 - val_loss: 2.4521
Epoch 10/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.3126 - val_loss: 2.4448
Epoch 11/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.1809 - val_loss: 2.4441
Epoch 12/100
108/108 ━━━━━━━━━━━━━━━━━━━

In [120]:
combined_data["strength"] = embed_surfer.predict(combined_data[["numeric_name", "numeric_event"]])

161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [121]:
combined_data[["Name", "event_group_name", "strength"]].sort_values(by="strength", ascending=False).drop_duplicates().head(50)

,Name,event_group_name,strength
1561,Filipe Toledo,Surf Ranch,16.306940
1563,Ethan Ewing,Surf Ranch,16.228735
3356,John John Florence,Surf Ranch,16.220579
1195,Gabriel Medina,Surf Ranch,16.096069
1562,Italo Ferreira,Surf Ranch,15.946934
3339,Jack Robinson,Surf Ranch,15.692001
2187,Filipe Toledo,El Salvador,15.414552
1094,Ethan Ewing,El Salvador,15.333879
2203,John John Florence,El Salvador,15.325463
814,Filipe Toledo,J-Bay Open,15.322148


: 

Now you could try to make the same thing but better using the functional API.

You could make it to take multiple input layers, that look at the nationality effect, the surfer effect, the DOB effect etc. and then each will output a probability and perhaps you can add the probabilities?

multiple inputs --> multiple dense output layers w/ probabolities ---> merge layer (add) --> dense output layer w/ prponbability



Skip connections - very easy to implement and seem very powerful.

Could create a model with two outputs, one which is a regression of the score of a surfer or score diff, and then one that is a win or loss. Embed the categorical and then concatenate with numeric inputs.

In [95]:
combined_data.columns

Index(['heat_id', 'round_num', 'round_name', 'gender', 'event_name',
       'event_group_name', 'wave_range', 'wind', 'date', 'tour_code',
       'surfer_id', 'name', 'heat_total', 'nationality', 'Name', 'Nation',
       'Followers', 'Age', 'Birthdate', 'Height', 'Weight', 'Hometown',
       'First Season', 'Stance', 'won', 'round_name_factor',
       'event_group_name_factor', 'wave_range_factor', 'wind_factor',
       'Name_factor', 'Nation_factor', 'Hometown_factor', 'Stance_factor',
       'numeric_name', 'strength', 'numeric_event'],
      dtype='object')